### 1. Load Data

In [9]:
import matplotlib.pyplot as plt

In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

/Users/hyunwoogu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


### 2. Parameters

In [2]:
# Training Parameters
learning_rate  = 0.001
training_steps = 1000
batch_size     = 128
display_step   = 200

# Network Parameters
num_input   = 28 # MNIST data input (img shape: 28*28)
timesteps   = 28 # timesteps
num_hidden  = 128 # hidden layer num of features
num_classes = 10 # MNIST total classes (0-9 digits)

### 3. Inputs

In [3]:
# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])

### 4. Weights and Biases

In [4]:
# Define weights
weights = {'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))}
biases  = {'out': tf.Variable(tf.random_normal([num_classes]))}

### 5. RNN

In [5]:
def RNN(x, weights, biases):
    x = tf.unstack(x, timesteps, 1)
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [6]:
logits     = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)
loss_op    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op  = optimizer.minimize(loss_op)

correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy     = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

### 6. Training

In [7]:
with tf.Session() as sess:
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x, Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.2476, Training Accuracy= 0.227
Step 200, Minibatch Loss= 0.3014, Training Accuracy= 0.875
Step 400, Minibatch Loss= 0.0634, Training Accuracy= 0.984
Step 600, Minibatch Loss= 0.1597, Training Accuracy= 0.930
Step 800, Minibatch Loss= 0.1301, Training Accuracy= 0.945
Step 1000, Minibatch Loss= 0.1273, Training Accuracy= 0.969
Optimization Finished!
Testing Accuracy: 1.0
